# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

H. Jiang  ->  H. Jiang  |  ['H. Jiang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
B. Maity  ->  B. Maity  |  ['B. Maity']
F. Davies  ->  F. Davies  |  ['F. Davies']
Arxiv has 67 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2505.09107


extracting tarball to tmp_2505.09107... done.
Retrieving document from  https://arxiv.org/e-print/2505.09148


extracting tarball to tmp_2505.09148...

 done.
Retrieving document from  https://arxiv.org/e-print/2505.09369


...

✔ → 12833:\section{Formalism}
  ↳ 37627:\section{Demonstration with Fiducial Model}


✔ → 37627:\section{Demonstration with Fiducial Model}
  ↳ 45161:\section{Observational Data \& Likelihood Analysis}


✔ → 45161:\section{Observational Data \& Likelihood Analysis}
  ↳ 52006:\section{Results \& Interpretations}\label{sec:results}


✔ → 52006:\section{Results \& Interpretations}\label{sec:results}
  ↳ 63446:\section{Summary \& Conclusions}\label{sec:summary}
✔ → 63446:\section{Summary \& Conclusions}\label{sec:summary}
  ↳ 67683:\section{Justification for Gaussian likelihood}
✔ → 67683:\section{Justification for Gaussian likelihood}
  ↳ 68549:end


Found 67 bibliographic references in tmp_2505.09369/aanda.bbl.
Issues with the citations
syntax error in line 345: '}' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.09369-b31b1b.svg)](https://arxiv.org/abs/2505.09369) | **Efficient Modelling of Lyman-α opacity fluctuations during late EoR**  |
|| <mark>B. Maity</mark>, <mark>F. Davies</mark>, P. Gaikwad |
|*Appeared on*| *2025-05-15*|
|*Comments*| *submitted to A&A*|
|**Abstract**|            The Lyman-$\alpha$ forest opacity fluctuations observed from high redshift quasar spectra have been proven to be extremely successful in order to probe the late phase of the reionization epoch. For ideal modeling of these opacity fluctuations, one of the main challenges is to satisfy the extremely high dynamic range requirements of the simulation box, resolving the Lyman alpha forest while probing the large cosmological scales. In this study, we adopt an efficient approach to model Lyman-$\alpha$ opacity fluctuations in coarse simulation volume, utilizing the semi-numerical reionization model SCRIPT (including inhomogeneous recombination and radiative feedback) integrated with a realistic photoionization background fluctuation generating model. Our model crucially incorporates ionization and temperature fluctuations, which are consistent with the reionization model. After calibrating our method with respect to high-resolution full hydrodynamic simulation, Nyx, we compare the models with available observational data at the redshift range, $z=5.0-6.1$. With a fiducial reionization model (reionization end at $z=5.8$), we demonstrate that the observed scatter in the effective optical depth can be matched reasonably well by tuning the free parameters of our model i.e. the effective ionizing photon mean free path and mean photoionization rate. We further pursue an MCMC-based parameter space exploration utilizing the available data to put constraints on the above free parameters. Our estimation prefers a slightly higher photoionization rate and slightly lower mean free path than the previous studies which is also a consequence of temperature fluctuations. This study holds significant promise for efficiently extracting important physical information about the Epoch of Reionization, utilizing the wealth of available and upcoming observational data.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.09107-b31b1b.svg)](https://arxiv.org/abs/2505.09107) | **Architecture of Tianyu Software: Relative Photometry as a Case Study**  |
|| Y. Rui, et al. -- incl., <mark>H. Jiang</mark> |
|*Appeared on*| *2025-05-15*|
|*Comments*| *18 pages, 10 figures, 6 tables, accepted for publication in PASP*|
|**Abstract**|            Tianyu telescope, an one-meter robotic optical survey instrument to be constructed in Lenghu, Qinghai, China, is designed for detecting transiting exoplanets, variable stars and transients. It requires a highly automated, optimally distributed, easily extendable, and highly flexible software to enable the data processing for the raw data at rates exceeding 500MB/s. In this work, we introduce the architecture of the Tianyu pipeline and use relative photometry as a case to demonstrate its high scalability and efficiency. This pipeline is tested on the data collected from Muguang observatory and Xinglong observatory. The pipeline demonstrates high scalability, with most processing stages increasing in throughput as the number of consumers grows. Compared to a single consumer, the median throughput of image calibration, alignment, and flux extraction increases by 41%, 257%, and 107% respectively when using 5 consumers, while image stacking exhibits limited scalability due to I/O constraints. In our tests, the pipeline was able to detect two transiting sources. Besides, the pipeline captures variability in the light curves of nine known and two previously unknown variable sources in the testing data. Meanwhile, the differential photometric precision of the light curves is near the theoretical limitation. These results indicate that this pipeline is suitable for detecting transiting exoplanets and variable stars. This work builds the fundation for further development of Tianyu software. Code of this work is available at this https URL.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2505.09148-b31b1b.svg)](https://arxiv.org/abs/2505.09148) | **VAST-MeMeS: Characterising non-thermal radio emission from magnetic massive stars using the Australian SKA Pathfinder**  |
|| B. Das, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-05-15*|
|*Comments*| *Submitted to PASA*|
|**Abstract**|            Magnetic massive stars are stars of spectral types O, B and A that harbour $\sim$ kG strength (mostly dipolar) surface magnetic fields. Their non-thermal radio emission has been demonstrated to be an important magnetospheric probe, provided the emission is fully characterised. A necessary step for that is to build a statistically significant sample of radio-bright magnetic massive stars. In this paper, we present the `VAST project to study Magnetic Massive Stars' or VAST-MeMeS that aims to achieve that by taking advantage of survey data acquired with the Australian SKA Pathfinder telescope. VAST-MeMeS is defined under the `VAriable and Slow Transient' (VAST) survey, although it also uses data from other ASKAP surveys. We found radio detections from 48 magnetic massive stars, out of which, 14 do not have any prior radio detections. We also identified 9 `Main-sequence Radio Pulse Emitter' candidates based on variability and circular polarisation of flux densities. The expanded sample suggests a lower efficiency in the radio production than that reported in earlier work. In addition to significantly expanding the sample of radio-bright magnetic massive stars, the addition of flux density measurements at $\lesssim 1$ GHz revealed that the spectra of incoherent radio emission can extend to much lower frequencies than that assumed in the past. In the future, radio observations spanning wide frequency and rotational phase ranges should be conducted so as to reduce the uncertainties in the incoherent radio luminosities. The results from these campaigns, supplemented with precise estimations of stellar parameters, will allow us to fully understand particle acceleration and non-thermal radio production in large-scale stellar magnetospheres.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2505.09369/./version_renew/paper_plots/Figure_3a_Delta_taupix_tau_eff_dist.png', 'tmp_2505.09369/./version_renew/paper_plots/fiducial_reion_model.png', 'tmp_2505.09369/./version_renew/paper_plots/Figure_6_tau_snap_gamma_temp.png']
copying  tmp_2505.09369/./version_renew/paper_plots/Figure_3a_Delta_taupix_tau_eff_dist.png to _build/html/
copying  tmp_2505.09369/./version_renew/paper_plots/fiducial_reion_model.png to _build/html/
copying  tmp_2505.09369/./version_renew/paper_plots/Figure_6_tau_snap_gamma_temp.png to _build/html/
exported in  _build/html/2505.09369.md
    + _build/html/tmp_2505.09369/./version_renew/paper_plots/Figure_3a_Delta_taupix_tau_eff_dist.png
    + _build/html/tmp_2505.09369/./version_renew/paper_plots/fiducial_reion_model.png
    + _build/html/tmp_2505.09369/./version_renew/paper_plots/Figure_6_tau_snap_gamma_temp.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\be}{\begin{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand\bear{#1}\ear{\begin{align}#1\end{align}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\TRC}[1]{{\color{red}[{\bf TRC:} #1]}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\AP}[1]{\textbfit{\color{purple}[AP: #1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\usepackage[T1]{fontenc}$
$\usepackage{amsmath}$
$\DeclareRobustCommand{\VAN}[3]{#2}$
$\let\VANthebibliography\thebibliography$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$
$\usepackage{graphicx}$
$\usepackage{txfonts}$
$\begin{document}$
$   \title{Efficient Modelling of Lyman-\alpha opacity fluctuations during late EoR}$
$    \authorrunning{Maity et al.}$
$   \author{Barun Maity\inst{1}\fnmsep\thanks{maity@mpia.de}$
$          ,$
$          Frederick Davies$
$          \inst{1},$
$          \and$
$          Prakash Gaikwad$
$          \inst{2}}$
$   \institute{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany$
$         \and Department of Astronomy, Astrophysics and Space Engineering, Indian Institute of Technology Indore, Simrol, MP 453552, India}$
$   \date{Received XXX; accepted XXX}$
$  \abstract{The Lyman-\alpha forest opacity fluctuations observed from high redshift quasar spectra have been proven to be extremely successful in order to probe the late phase of the reionization epoch. For ideal modeling of these opacity fluctuations, one of the main challenges is to satisfy the extremely high dynamic range requirements of the simulation box,  resolving the Lyman alpha forest while probing the large cosmological scales. In this study,  we adopt an efficient approach to model Lyman-\alpha opacity fluctuations in coarse simulation volume, utilizing the semi-numerical reionization model SCRIPT (including inhomogeneous recombination and radiative feedback) integrated with a realistic photoionization background fluctuation generating model. Our model crucially incorporates ionization and temperature fluctuations, which are consistent with the reionization model. After calibrating our method with respect to high-resolution full hydrodynamic simulation, Nyx, we compare the models with available observational data at the redshift range, z=5.0-6.1. With a fiducial reionization model (reionization end at z=5.8), we demonstrate that the observed scatter in the effective optical depth can be matched reasonably well by tuning the free parameters of our model i.e. the effective ionizing photon mean free path and mean photoionization rate. We further pursue an MCMC-based parameter space exploration utilizing the available data to put constraints on the above free parameters. Our estimation prefers a slightly higher photoionization rate and slightly lower mean free path than the previous studies which is also a consequence of temperature fluctuations. This study holds significant promise for efficiently extracting important physical information about the Epoch of Reionization, utilizing the wealth of available and upcoming observational data.$
$}$
$   \keywords{intergalactic medium -- cosmology: theory – dark ages, reionization, first stars -- large-scale structure of Universe}$
$   \maketitle$
$\n\end{document}\end{equation}}$
$\newcommand{\ee}{\end{equation}}$
$\newcommand{\nline}{\notag \\}$
$\newcommand{\f}{\frac}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\del}{\partial}$
$\newcommand{\half}{\frac{1}{2}}$
$\newcommand{\im}{\mathrm{i}}$
$\newcommand{\e}{\mathrm{e}}$
$\newcommand{\Msun}{\mathrm{M}_{\odot}}$
$\newcommand{\eqn}[1]{equation~(\ref{#1})}$
$\newcommand{\eqns}[2]{equations~(\ref{#1}) and~(\ref{#2})}$
$\newcommand{\secn}[1]{Section~\ref{#1}}$
$\newcommand{\appndx}[1]{Appendix~\ref{#1}}$
$\newcommand{\fig}[1]{Fig.~\ref{#1}}$
$\newcommand{\figs}[1]{Figs.~\ref{#1}}$
$\newcommand{\tab}[1]{Table~\ref{#1}}$
$\newcommand{\TRC}[1]{{\color{red}[{\bf TRC:} #1]}}$
$\newcommand{\BM}[1]{{\color{blue}[{\bf }#1]}}$
$\newcommand{\AP}[1]{\textbfit{\color{purple}[AP: #1]}}$
$\newcommand{\red}[1]{{\color{red} #1}}$
$\newcommand{\arraystretch}{1.2}$
$\newcommand\bear{#1}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Efficient Modelling of Lyman-$\alpha$ opacity fluctuations during late EoR

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2505.09369-b31b1b.svg)](https://arxiv.org/abs/2505.09369)<mark>Appeared on: 2025-05-15</mark> -  _submitted to A&A_

</div>
<div id="authors">

<mark>B. Maity</mark>, <mark>F. Davies</mark>, P. Gaikwad

</div>
<div id="abstract">

**Abstract:** The Lyman- $\alpha$ forest opacity fluctuations observed from high redshift quasar spectra have been proven to be extremely successful in order to probe the late phase of the reionization epoch. For ideal modeling of these opacity fluctuations, one of the main challenges is to satisfy the extremely high dynamic range requirements of the simulation box,  resolving the Lyman alpha forest while probing the large cosmological scales. In this study,  we adopt an efficient approach to model Lyman- $\alpha$ opacity fluctuations in coarse simulation volume, utilizing the semi-numerical reionization model SCRIPT (including inhomogeneous recombination and radiative feedback) integrated with a realistic photoionization background fluctuation generating model. Our model crucially incorporates ionization and temperature fluctuations, which are consistent with the reionization model. After calibrating our method with respect to high-resolution full hydrodynamic simulation, Nyx, we compare the models with available observational data at the redshift range, $z=5.0-6.1$ . With a fiducial reionization model (reionization end at $z=5.8$ ), we demonstrate that the observed scatter in the effective optical depth can be matched reasonably well by tuning the free parameters of our model i.e. the effective ionizing photon mean free path and mean photoionization rate. We further pursue an MCMC-based parameter space exploration utilizing the available data to put constraints on the above free parameters. Our estimation prefers a slightly higher photoionization rate and slightly lower mean free path than the previous studies which is also a consequence of temperature fluctuations. This study holds significant promise for efficiently extracting important physical information about the Epoch of Reionization, utilizing the wealth of available and upcoming observational data.

</div>

<div id="div_fig1">

<img src="tmp_2505.09369/./version_renew/paper_plots/Figure_3a_Delta_taupix_tau_eff_dist.png" alt="Fig6" width="100%"/>

**Figure 6. -** _Left Panel:_ The distribution of densities ($\log \Delta$) from the hydro simulation (blue) and the semi-numerical setup (dashed orange). _Middle Panel:_ The distribution of optical depths averaged at $4h^{-1}\mathrm{cMpc}$ scale ($\tau_i$) for hydro simulation (blue), calibrated empirical relation (green), and calibrated semi-numerical setup (dashed orange). _Right Panel:_ The distributions of effective optical depth skewers ($\tau_{\mathrm{eff}}$) for the previous three cases. (*fig:fig4_tau_delta_dist*)

</div>
<div id="div_fig2">

<img src="tmp_2505.09369/./version_renew/paper_plots/fiducial_reion_model.png" alt="Fig7" width="100%"/>

**Figure 7. -** The figure shows the neutral fraction ($Q_{\mathrm{HI}}^V$), mean IGM temperature ($T_0$), and $T-\Delta$ power law index ($\gamma$) evolution with redshift for our fiducial reionization model (_blue solid_ lines) along with other two variants using different reionization temperature increment, $T_{\mathrm{re}}$(high: _green dashed_; low: _red dashed_). We also show the recent observational constraints on the neutral fractions \citep{2018ApJ...864..142D,2022MNRAS.512.5390G,2024MNRAS.533L..49Z,2024A&A...688L..26S} and IGM temperatures \citep{2020MNRAS.494.5091G}. (*fig:fig5_fid_model*)

</div>
<div id="div_fig3">

<img src="tmp_2505.09369/./version_renew/paper_plots/Figure_6_tau_snap_gamma_temp.png" alt="Fig9" width="100%"/>

**Figure 9. -** The snapshots of Ly-$\alpha$ optical depth for different combinations of temperature and UVB fluctuation inputs. From left to right, we show the cases gradually where we include i) no UVB/temperature fluctuation; ii) only temperature fluctuation but no UVB fluctuations; iii) weak UVB fluctuations with temperature fluctuations, and iv) strong UVB fluctuations with temperature fluctuations. (*fig:fig7_tau_snapshots*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2505.09369"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

489  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

13  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
